In [2]:
import csv
import pandas as pd
import re

if __name__=='__main__':
#Retriving the Headers from the example.csv file
    with open('example.csv', 'r', encoding="utf-8") as f:
        f_reader = csv.reader(f)
        header =[]
        header = next(f_reader)
        
    #Renaming the headers from homework.csv using pandas and saving it to new homework1.csv to retain original homework.csv file
    df = pd.read_csv("homework.csv", encoding="utf-8")

    df.rename(columns={"item weight (pounds)": "weight",
                       'item depth (inches)':'length',
                       'item height (inches)':'height',
                       'item width (inches)':'width', 
                       'wholesale ($)':'cost_price',
                       'map ($)':'min_price',
                       'description':'product_title',
                       'long description':'product_description',
                       'brand':'product__brand__name',
                       'item category':'product__product_class__name',
                       'item materials':'attrib__material',
                       'item finish':'attrib__finish',
                       'bulb 1 count':'attrib__number_bulbs',
                       'bulb 1 type':'attrib__bulb_type',
                       'switch type':'attrib__switch_type',
                       'bulb 1 wattage':'attrib__wattage',
                       'outdoor':'attrib__outdoor_safe',
                       'primary color family':'attrib__color',
                       'carton 1 width (inches)':'boxes__0__width',
                       'carton 1 length (inches)':'boxes__0__length',
                       'carton 1 height (inches)':'boxes__0__height',
                       'carton 1 weight (pounds)':'boxes__0__weight',
                       'carton 2 width (inches)':'boxes__1__width',
                       'carton 2 length (inches)':'boxes__1__length',
                       'carton 2 height (inches)':'boxes__1__height',
                       'carton 2 weight (pounds)':'boxes__1__weight',
                       'carton 3 width (inches)':'boxes__2__width',
                       'carton 3 length (inches)':'boxes__2__length',
                       'carton 3 height (inches)':'boxes__2__height',
                       'carton 3 weight (pounds)':'boxes__2__weight',
                       'conversion kit option':'attrib__kit',
                       'safety rating':'attrib__ul_certified', #convert UL to YES else to NO
                       'cord length (inches)':'attrib__cord_length',
                       'furniture arm height (inches)': 'attrib__arm_height',
                       'furniture seat height (inches)':'attrib__seat_height',
                       'furniture seat dimensions (inches)':'attrib__seat_width', #change
                       'furniture weight capacity (pounds)':'attrib__weight_capacity',
                       'country of origin':'product__country_of_origin__alpha_3',
                       'product_title' : 'product__title',
                       'long description' : 'product__description',
                      'item style':'product__styles',
                       'upc':'ean13',
                      'url california label (jpg)':'prop_65',}, inplace=True)

    df.to_csv('homework1.csv', index=False)
    
    #creating a list with the updated Headers from homework(1).csv
    with open('homework1.csv', 'r', encoding="utf-8") as e:
        e_reader = csv.reader(e)
        header2 =[]
        header2 = next(e_reader)
        
    #creating a list with Headers that were common to both example.csv and UPDATED homework(1).csv
    common = list(set(header).intersection(header2))

    #creating new csv file and removing datasets not present/required according to example.csv, saving to new homework2.csv
    df2 = pd.read_csv("homework1.csv", encoding="utf-8")
    keep_col = common
    dfChanged = df2[keep_col]
    dfChanged.to_csv('homework2.csv', index=False)
    
#filling up the rows of homework(3).csv with data
    with open('homework2.csv', 'r', encoding="utf-8") as g:
        g_reader = csv.DictReader(g)
        #next(g_reader)
        with open('homework3.csv', 'w', encoding="utf-8") as h:
            csv_writer = csv.DictWriter(h, header, lineterminator ='\n')
            csv_writer.writeheader()
            csv_writer.writerows(g_reader)
            
   #making final adjustments to some values in attrib__ul_certified, product__country_of_origin__alpha_3 and attrib__seat_width columns
    dframe = pd.read_csv("homework3.csv")
    a = len(dframe)

    #changing from float to string to add 'Yes/NO' values
    dframe['attrib__distressed_finish'] = dframe['attrib__distressed_finish'].map(str)
    dframe['prop_65'] = dframe['prop_65'].map(str)
    dframe['ean13'] = dframe['ean13'].map(str)

    for numb in range(0,a):

        #Changing UL certified from 'UL' to either 'YES' or 'NO'
        if dframe.at[numb,'attrib__ul_certified'] == 'UL': 
            dframe.at[numb,'attrib__ul_certified'] = 'YES'
        else:
            dframe.at[numb,'attrib__ul_certified'] = 'NO'    

        #Changing name of Countries to 3 digit ALPHA
        word = (dframe.at[numb,'product__country_of_origin__alpha_3'])
        if type(word) == str:
            (dframe.at[numb,'product__country_of_origin__alpha_3']) = word[0:3].upper()

        seat_width = (dframe.at[numb,'attrib__seat_width'])
        if type(seat_width) == str:
            dframe.at[numb,'attrib__seat_width']= seat_width.split('x')[0]

        #removing '$' signs and ',' from prices to round off to 2 decimal
        cost = str(dframe.at[numb,'cost_price'])
        cost = re.sub('[$,]', '', cost)
        cost = float(cost)
        dframe.at[numb,'cost_price'] = round(cost, 2)

        mini = str(dframe.at[numb,'min_price'])
        mini = re.sub('[$,]', '', mini)
        mini = float(mini)
        dframe.at[numb,'cost_price'] = round(mini, 2)

        #adding Yes/No values to attrib__distressed_finish
        fullstring = str(dframe.at[numb,'product__description'])
        substring = "[Dd]istress"

        if re.search(substring, fullstring):
            dframe.at[numb,'attrib__distressed_finish'] = 'Yes' 
        else:
            dframe.at[numb,'attrib__distressed_finish'] = 'No'

        #formatting upc to ean using simple if and while loops
        ean = str(dframe.at[numb,'ean13'])
        if ean != '': 
            n = 14 - len(ean)
            while len(ean)<15 and ean != '':
                ean = '0' + ean
        ean = ean.split('.')[0]
        ean = ean[:3] + '-' + ean[3:]
        ean = ean[:-1] + '-' + ean[-1:]
        dframe.at[numb,'ean13'] = ean

        #prop 65
        if  dframe.at[numb,'prop_65'] != '':
                dframe.at[numb,'prop_65'] = 'TRUE'
        else:
                dframe.at[numb,'prop_65'] = 'FALSE'

    dframe.to_csv('formatted.csv', index=False) 

081-093700203-5
081-093700205-9
081-093700208-0
081-093700210-3
081-093700217-2
081-093700239-4
081-093700248-6
081-093700271-4
081-093700276-9
081-093700281-3
081-093700285-1
081-093700287-5
081-093700289-9
081-093700316-2
081-093700319-3
081-093700340-7
081-093700352-0
081-093700401-5
081-093700402-2
081-093700444-2
081-093700500-5
081-093700505-0
081-093700590-6
081-093700671-2
081-093700704-7
081-093700722-1
081-093700729-0
081-093700801-3
081-093700807-5
081-093700992-8
081-093702002-2
081-093708000-2
081-093708001-9
081-093708004-0
081-093708005-7
081-093708011-8
081-093710005-2
084-355814057-2
084-355814056-5
084-355814055-8
080-432300005-7
080-432301525-8
080-432302506-7
080-432310250-8
080-432301526-6
080-432310300-0
080-432303007-8
080-432304912-4
080-432304960-5
080-432304004-6
080-432304006-0
080-432304506-5
080-432304510-2
080-434318929-9
080-434319048-6
080-434319051-6
080-434319052-3
080-434319053-0
080-434319061-5
080-434318660-1
080-434318661-8
081-800801739-1
081-8008

084-355815663-4
084-355815664-1
084-355815665-8
084-355815666-5
084-355815667-2
084-355815668-9
084-355815669-6
084-355815670-2
084-355815671-9
084-355815672-6
084-355815673-3
081-800803898-3
081-800803899-0
081-800803900-3
081-800803902-7
081-800803903-4
081-800803904-1
081-800803905-8
081-800803906-5
081-800803907-2
081-800804325-3
081-800804326-0
081-800803758-0
081-800803759-7
081-800803760-3
081-800803761-0
084-355815967-3
084-355815968-0
084-355815969-7
084-355815970-3
084-355815971-0
084-355815972-7
084-355815973-4
084-355815974-1
084-355815975-8
084-355815976-5
084-355815565-1
084-355815566-8
084-355815567-5
084-355815568-2
084-355815569-9
084-355815570-5
084-355815571-2
084-355815572-9
084-355815573-6
084-355815574-3
081-800803747-4
084-355815575-0
084-355815576-7
084-355815577-4
084-355815578-1
084-355815579-8
084-355815580-4
084-355815581-1
084-355815682-5
084-355815683-2
084-355815684-9
084-355815686-3
084-355815688-7
084-355815689-4
084-355815690-0
084-355815691-7
084-3558

081-800804268-3
081-800804271-3
084-355816619-0
081-800804270-6
081-800804272-0
081-800804273-7
081-800804267-6
081-800804527-1
084-355816625-1
084-355816626-8
084-355816627-5
084-355816632-9
084-355816631-2
084-355816620-6
084-355816638-1
084-355816639-8
084-355816621-3
084-355816641-1
084-355816635-0
084-355816624-4
084-355816642-8
084-355816622-0
084-355816628-2
084-355816623-7
084-355816637-4
084-355816640-4
084-355816633-6
084-355816634-3
084-355816629-9
084-355816636-7
084-355816630-5
000-000000000na-n
000-000000000na-n
000-000000000na-n
000-000000000na-n
000-000000000na-n
000-000000000na-n
000-000000000na-n
000-000000000na-n
000-000000000na-n
000-000000000na-n
000-000000000na-n
000-000000000na-n
000-000000000na-n
000-000000000na-n
000-000000000na-n
000-000000000na-n
000-000000000na-n
083-033502493-4
083-033502530-6
080-434319135-3
080-434319138-4
080-434319141-4
080-434319142-1
080-434319144-5
080-434319145-2
084-355813738-1
084-355813739-8
084-355813740-4
084-355813741-1
084-35

081-800803353-7
084-355815331-2
084-355815332-9
081-800803355-1
081-800803356-8
081-800804344-4
081-800804345-1
081-800804346-8
081-800804347-5
081-800804348-2
081-800804349-9
081-800804350-5
081-800804498-4
081-800804499-1
081-800804500-4
081-800804501-1
081-800804502-8
084-355816686-2
081-800804503-5
084-355813058-0
084-355813059-7
084-355813060-3
084-355813062-7
084-140600910-1
084-140601066-4
084-140601068-8
084-355801539-9
084-355800683-0
084-355800916-9
084-140601942-1
084-355800489-8
081-800801726-1
081-800801727-8
081-800801728-5
081-800801729-2
081-800801731-5
081-800801732-2
081-800801733-9
081-800801734-6
081-800801730-8
081-800801736-0
081-800801737-7
081-800801735-3
084-355804792-5
084-355804794-9
084-355802839-9
084-355802840-5
084-355802847-4
084-140600538-7
081-800802243-2
081-800802244-9
081-800802245-6
081-800802246-3
081-800802247-0
081-800802249-4
081-800802250-0
081-800802251-7
084-140601684-0
084-140601682-6
084-355801717-1
084-355801945-8
081-800803328-5
084-3558

074-811907395-8
074-811903877-3
074-811907397-2
074-811907400-9
074-811908005-5
074-811907496-2
083-033502690-7
083-033503045-4
074-811908076-5
074-811907497-9
083-033502691-4
083-033503046-1
074-811908077-2
074-811907498-6
083-033502692-1
083-033503047-8
074-811908078-9
081-800802523-5
083-033502693-8
083-033503048-5
074-811909495-3
074-811908116-8
074-811903878-0
081-800802524-2
083-033502694-5
083-033503049-2
074-811909494-6
081-800802532-7
074-811909278-2
083-033502695-2
083-033503050-8
074-811909493-9
074-811908117-5
074-811909279-9
083-033502696-9
083-033503051-5
074-811909492-2
074-811909283-6
074-811909488-5
074-811909284-3
074-811909487-8
074-811909285-0
074-811909486-1
074-811909286-7
083-033502698-3
083-033503052-2
074-811909485-4
074-811909287-4
074-811909484-7
074-811909288-1
083-033502699-0
083-033503053-9
074-811909483-0
074-811909289-8
074-811909482-3
074-811909298-0
074-811909473-1
074-811909299-7
074-811909472-4
074-811909300-0
074-811909471-7
074-811909301-7
083-0335